<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Testing different Hyperparameters and Benchmarking

In this notebook, we will cover how to test different hyperparameters for a particular dataset and how to benchmark different parameters across a group of datasets using AzureML. We assume familiarity with the basic concepts and parameters, which are discussed in the [01_training_introduction.ipynb](01_training_introduction.ipynb), [02_mask_rcnn.ipynb](02_mask_rcnn.ipynb) and [03_training_accuracy_vs_speed.ipynb](03_training_accuracy_vs_speed.ipynb) notebooks. 

We will be using a Faster R-CNN model with ResNet-50 backbone to find all objects in an image belonging to 4 categories: 'can', 'carton', 'milk_bottle', 'water_bottle'. We will then conduct hyper-parameter tuning to find the best set of parameters for this model. For this, we present an overall process of utilizing AzureML, specifically [Hyperdrive](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive?view=azure-ml-py) which can train and evaluate many different parameter combinations in parallel. We demonstrate the following key steps:  
* Configure AzureML Workspace
* Create Remote Compute Target (GPU cluster)
* Prepare Data
* Prepare Training Script
* Setup and Run Hyperdrive Experiment
* Model Import, Re-train and Test

This notebook is very similar to the [24_exploring_hyperparameters_on_azureml.ipynb](../../classification/notebooks/24_exploring_hyperparameters_on_azureml.ipynb) hyperdrive notebook used for image classification. For key concepts of AzureML see this [tutorial](https://docs.microsoft.com/en-us/azure/machine-learning/service/tutorial-train-models-with-aml?view=azure-ml-py&toc=https%3A%2F%2Fdocs.microsoft.com%2Fen-us%2Fpython%2Fapi%2Fazureml_py_toc%2Ftoc.json%3Fview%3Dazure-ml-py&bc=https%3A%2F%2Fdocs.microsoft.com%2Fen-us%2Fpython%2Fazureml_py_breadcrumb%2Ftoc.json%3Fview%3Dazure-ml-py) on model training and evaluation.

In [1]:
import os
import sys
from distutils.dir_util import copy_tree
import numpy as np
import scrapbook as sb
import uuid

import azureml.core
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
import azureml.data
from azureml.train.estimator import Estimator
from azureml.train.hyperdrive import (
    RandomParameterSampling, GridParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal, choice, uniform
)
import azureml.widgets as widgets

sys.path.append("../../")
from utils_cv.common.data import unzip_url
from utils_cv.detection.data import Urls

Ensure edits to libraries are loaded and plotting is shown in the notebook.

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

We now define some parameters which will be used in this notebook:

In [44]:
# Azure resources
#subscription_id = "YOUR_SUBSCRIPTION_ID"
#resource_group = "YOUR_RESOURCE_GROUP_NAME"  
#workspace_name = "YOUR_WORKSPACE_NAME"  
#workspace_region = "YOUR_WORKSPACE_REGION" #Possible values eastus, eastus2, etc.

###subscription_id = "2ad17db4-e26d-4c9e-999e-adae9182530c"  #Sharat 
subscription_id = "989b90f7-da4f-41f9-84c9-44848802052d"  #E2E
resource_group = "pabuehle_delme_hyperdrive"  
workspace_name = "pabuehle_ws"  
workspace_region = "southcentralus" #Possible values eastus, eastus2, etc.

# Choose a size for our cluster and the maximum number of nodes
VM_SIZE = "STANDARD_NC6" #"STANDARD_NC6", STANDARD_NC6S_V3"
MAX_NODES = 10

# Hyperparameter grid search space
IM_MAX_SIZES = [600] #Default is 1333 pixels, defining small values here to speed up training
LEARNING_RATES = [1e-4, 3e-4, 1e-3, 3e-3, 1e-2, 3e-2, 1e-1, 3e-1]
print(LEARNING_RATES)
print(len(LEARNING_RATES))

# Image data
#DATA_PATH = unzip_url(Urls.fridge_objects_path, exist_ok=True)
#DATA_PATH = unzip_url("https://cvbp.blob.core.windows.net/public/datasets/object_detection/odRefrigerator.zip", exist_ok=True)

#DATA_PATH = unzip_url("https://cvbp.blob.core.windows.net/public/datasets/object_detection/odCokeBottle.zip", exist_ok=True)
#DATA_PATH = unzip_url("https://cvbp.blob.core.windows.net/public/datasets/object_detection/odDrone.zip", exist_ok=True)
#DATA_PATH = unzip_url("https://cvbp.blob.core.windows.net/public/datasets/object_detection/odGiraffe.zip", exist_ok=True)
#DATA_PATH = unzip_url("https://cvbp.blob.core.windows.net/public/datasets/object_detection/odPipe.zip", exist_ok=True)
#DATA_PATH = unzip_url("https://cvbp.blob.core.windows.net/public/datasets/object_detection/odRefrigerator.zip", exist_ok=True)
DATA_PATH = unzip_url("https://cvbp.blob.core.windows.net/public/datasets/object_detection/odTorso.zip", exist_ok=True)
#DATA_PATH = unzip_url("https://cvbp.blob.core.windows.net/public/datasets/object_detection/odVocSubset100th.zip", exist_ok=True)


[0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3]
8


### 1. Config AzureML workspace
Below we setup (or load an existing) AzureML workspace, and get all its details as follows. Note that the resource group and workspace will get created if they do not yet exist. For more information regaring the AzureML workspace see also the [20_azure_workspace_setup.ipynb](../../classification/notebooks/20_azure_workspace_setup.ipynb) notebook in the image classification folder.

To simplify clean-up (see end of this notebook), we recommend creating a new resource group to run this notebook.

In [4]:
from utils_cv.common.azureml import get_or_create_workspace

ws = get_or_create_workspace(
        subscription_id,
        resource_group,
        workspace_name,
        workspace_region)

# Print the workspace attributes
print('Workspace name: ' + ws.name, 
      'Workspace region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

ERROR - get_workspace error using subscription_id=989b90f7-da4f-41f9-84c9-44848802052d, resource_group_name=pabuehle_delme_hyperdrive, workspace_name=pabuehle_ws


Creating new workspace


Deploying KeyVault with name pabuehlekeyvault821a1637.
Deploying StorageAccount with name pabuehlestoragea7be9f73c.
Deploying AppInsights with name pabuehleinsights65c8581b.
Deployed AppInsights with name pabuehleinsights65c8581b. Took 28.67 seconds.
Deployed StorageAccount with name pabuehlestoragea7be9f73c. Took 58.36 seconds.
Deployed KeyVault with name pabuehlekeyvault821a1637. Took 52.26 seconds.
Deploying Workspace with name pabuehle_ws.
Deployed Workspace with name pabuehle_ws. Took 98.26 seconds.
Workspace name: pabuehle_ws
Workspace region: southcentralus
Subscription id: 989b90f7-da4f-41f9-84c9-44848802052d
Resource group: pabuehle_delme_hyperdrive


### 2. Create Remote Target
We create a GPU cluster as our remote compute target. If a cluster with the same name already exists in our workspace, the script will load it instead. This [link](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-set-up-training-targets#compute-targets-for-training) provides more information about how to set up a compute target on different locations.

By default, the VM size is set to use STANDARD\_NC6 machines. However, if quota is available, our recommendation is to use STANDARD\_NC6S\_V3 machines which come with the much faster V100 GPU. We set the minimum number of nodes to zero so that the cluster won't incur additional compute charges when not in use.

In [9]:
CLUSTER_NAME = "gpu-cluster"

try:
    # Retrieve if a compute target with the same cluster name already exists
    compute_target = ComputeTarget(workspace=ws, name=CLUSTER_NAME)
    print('Found existing compute target.')
    
except ComputeTargetException:
    # If it doesn't already exist, we create a new one with the name provided
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size=VM_SIZE,
                                                           min_nodes=0,
                                                           max_nodes=MAX_NODES)

    # create the cluster
    compute_target = ComputeTarget.create(ws, CLUSTER_NAME, compute_config)
    compute_target.wait_for_completion(show_output=True)

# we can use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Creating a new compute target...
Creating
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-09-16T18:39:03.768000+00:00', 'errors': None, 'creationTime': '2019-09-16T18:39:01.504602+00:00', 'modifiedTime': '2019-09-16T18:39:17.093939+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 10, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


The compute cluster and its status can be seen in the portal. For example in the screenshot below, its automatically resizing (eventually to 0 nodes) to adjust to the number of open runs:
<img src="media/hyperdrive_cluster.jpg" width="800" alt="Compute cluster status">

### 3. Prepare data
In this notebook, we'll use the Fridge Objects dataset, which is already stored in the correct format. We then upload our data to the AzureML workspace.


In [10]:
# Retrieving default datastore that got automatically created when we setup a workspace
ds = ws.get_default_datastore()

# We now upload the data to a unique sub-folder to avoid accidentially training/evaluating also including older images.
data_subfolder = str(uuid.uuid4())
ds.upload(
    src_dir=DATA_PATH,
    target_path=data_subfolder,
    overwrite=False,
    show_progress=True
)

Uploading an estimated of 60 files
Uploading C:\Users\pabuehle\Desktop\ComputerVision\data\odRefrigerator\annotations\0.xml
Uploading C:\Users\pabuehle\Desktop\ComputerVision\data\odRefrigerator\annotations\10.xml
Uploading C:\Users\pabuehle\Desktop\ComputerVision\data\odRefrigerator\annotations\11.xml
Uploading C:\Users\pabuehle\Desktop\ComputerVision\data\odRefrigerator\annotations\12.xml
Uploading C:\Users\pabuehle\Desktop\ComputerVision\data\odRefrigerator\annotations\13.xml
Uploaded C:\Users\pabuehle\Desktop\ComputerVision\data\odRefrigerator\annotations\0.xml, 1 files out of an estimated total of 60
Uploading C:\Users\pabuehle\Desktop\ComputerVision\data\odRefrigerator\annotations\14.xml
Uploaded C:\Users\pabuehle\Desktop\ComputerVision\data\odRefrigerator\annotations\10.xml, 2 files out of an estimated total of 60
Uploading C:\Users\pabuehle\Desktop\ComputerVision\data\odRefrigerator\annotations\15.xml
Uploaded C:\Users\pabuehle\Desktop\ComputerVision\data\odRefrigerator\annotat

Uploading C:\Users\pabuehle\Desktop\ComputerVision\data\odRefrigerator\images\21.jpg
Uploaded C:\Users\pabuehle\Desktop\ComputerVision\data\odRefrigerator\images\19.jpg, 40 files out of an estimated total of 60
Uploading C:\Users\pabuehle\Desktop\ComputerVision\data\odRefrigerator\images\22.jpg
Uploaded C:\Users\pabuehle\Desktop\ComputerVision\data\odRefrigerator\images\2.jpg, 41 files out of an estimated total of 60
Uploading C:\Users\pabuehle\Desktop\ComputerVision\data\odRefrigerator\images\23.jpg
Uploaded C:\Users\pabuehle\Desktop\ComputerVision\data\odRefrigerator\images\20.jpg, 42 files out of an estimated total of 60
Uploading C:\Users\pabuehle\Desktop\ComputerVision\data\odRefrigerator\images\24.jpg
Uploading C:\Users\pabuehle\Desktop\ComputerVision\data\odRefrigerator\images\25.jpg
Uploading C:\Users\pabuehle\Desktop\ComputerVision\data\odRefrigerator\images\26.jpg
Uploaded C:\Users\pabuehle\Desktop\ComputerVision\data\odRefrigerator\images\23.jpg, 43 files out of an estimated

$AZUREML_DATAREFERENCE_e6bc2ba50dd44047b27249278194b948


Here's where you can see the data in your portal: 
<img src="media/datastore.jpg" width="800" alt="Datastore screenshot for Hyperdrive notebook run">

### 4. Prepare training script

Next step is to prepare scripts that AzureML Hyperdrive will use to train and evaluate models with selected hyperparameters.

In [11]:
# Create a folder for the training script and copy the utils_cv library into that folder
script_folder = os.path.join(os.getcwd(), "hyperdrive")
os.makedirs(script_folder, exist_ok=True)
_ = copy_tree(os.path.join('..', '..', 'utils_cv'), os.path.join(script_folder, 'utils_cv'))

In [45]:
%%writefile $script_folder/train.py

# Use different matplotlib backend to avoid error during remote execution
import matplotlib 
matplotlib.use("Agg") 
import matplotlib.pyplot as plt

import os
import sys
import argparse
import numpy as np
from pathlib import Path
from azureml.core import Run
from utils_cv.detection.dataset import DetectionDataset
from utils_cv.detection.model import DetectionLearner, get_pretrained_fasterrcnn
from utils_cv.common.gpu import which_processor
which_processor()


# Parse arguments passed by Hyperdrive
parser = argparse.ArgumentParser()
parser.add_argument('--data-folder', type=str, dest='data_dir')
parser.add_argument('--data-subfolder', type=str, dest='data_subfolder')
parser.add_argument('--epochs', type=int, dest='epochs', default=30)                                     
parser.add_argument('--batch_size', type=int, dest='batch_size', default=2)
parser.add_argument('--learning_rate', type=float, dest='learning_rate', default=1e-4)
parser.add_argument('--min_size', type=int, dest='min_size', default=800)
parser.add_argument('--max_size', type=int, dest='max_size', default=1333)
parser.add_argument('--rpn_pre_nms_top_n_train', type=int, dest='rpn_pre_nms_top_n_train', default=2000)
parser.add_argument('--rpn_pre_nms_top_n_test', type=int, dest='rpn_pre_nms_top_n_test', default=1000)
parser.add_argument('--rpn_post_nms_top_n_train', type=int, dest='rpn_post_nms_top_n_train', default=2000)
parser.add_argument('--rpn_post_nms_top_n_test', type=int, dest='rpn_post_nms_top_n_test', default=1000)
parser.add_argument('--rpn_nms_thresh', type=float, dest='rpn_nms_thresh', default=0.7)
parser.add_argument('--box_score_thresh', type=float, dest='box_score_thresh', default=0.05)
parser.add_argument('--box_nms_thresh', type=float, dest='box_nms_thresh', default=0.5)
parser.add_argument('--box_detections_per_img', type=int, dest='box_detections_per_img', default=100)
args = parser.parse_args()
params = vars(args)
print(f"params = {params}")


# Perform up to 3 repetitions for small datasets
data_path = os.path.join(params['data_dir'], params["data_subfolder"])
print(f"data_path={data_path}")
nr_train_imgs = len(DetectionDataset(data_path, train_pct=0.5, batch_size = params["batch_size"]).train_ds)
if nr_train_imgs<150:
    nr_reps=3
elif nr_train_imgs<300:
    nr_reps=2
else:
    nr_reps=1

    
# Run model training and evaluation
accs = []
for rep_nr in range(nr_reps):
    print(f"**** RUNNING REPETITION {rep_nr} ****")

    # Getting training and validation data
    data = DetectionDataset(data_path, train_pct=0.5, batch_size = params["batch_size"])
    print(
        f"Training dataset: {len(data.train_ds)} | Training DataLoader: {data.train_dl} \n \
        Testing dataset: {len(data.test_ds)} | Testing DataLoader: {data.test_dl}"
    )

    # Get model
    model = get_pretrained_fasterrcnn(
        num_classes = len(data.labels)+1,
        min_size = params["min_size"],
        max_size = params["max_size"],
        rpn_pre_nms_top_n_train = params["rpn_pre_nms_top_n_train"],
        rpn_pre_nms_top_n_test = params["rpn_pre_nms_top_n_test"],
        rpn_post_nms_top_n_train = params["rpn_post_nms_top_n_train"], 
        rpn_post_nms_top_n_test = params["rpn_post_nms_top_n_test"],
        rpn_nms_thresh = params["rpn_nms_thresh"],
        box_score_thresh = params["box_score_thresh"], 
        box_nms_thresh = params["box_nms_thresh"],
        box_detections_per_img = params["box_detections_per_img"]
    )
    detector = DetectionLearner(data, model)

    # Run Training
    detector.fit(params["epochs"], lr=params["learning_rate"], print_freq=30)
    print(f"Average precision after each epoch: {detector.ap}")
    
    accs.append(float(detector.ap[-1]))
    print("Accuracies for each repetition: " + str(accs))
    
# Add log entries
run = Run.get_context()

run.log("accuracy", float(np.mean(accs)))  # Logging our primary metric 'accuracy'
run.log("accs", str(accs))  # Logging our primary metric 'accuracy'

#run.log("accuracy", float(detector.ap[-1]))  # Logging our primary metric 'accuracy'
run.log("data_dir", params["data_dir"])
run.log("epochs", params["epochs"])
run.log("batch_size", params["batch_size"])
run.log("learning_rate", params["learning_rate"])
run.log("min_size", params["min_size"])
run.log("max_size", params["max_size"])
run.log("rpn_pre_nms_top_n_train", params["rpn_pre_nms_top_n_train"])
run.log("rpn_pre_nms_top_n_test", params["rpn_pre_nms_top_n_test"])
run.log("rpn_post_nms_top_n_train", params["rpn_post_nms_top_n_train"])
run.log("rpn_post_nms_top_n_test", params["rpn_post_nms_top_n_test"])
run.log("rpn_nms_thresh", params["rpn_nms_thresh"])
run.log("box_score_thresh", params["box_score_thresh"])
run.log("box_nms_thresh", params["box_nms_thresh"])
run.log("box_detections_per_img", params["box_detections_per_img"])

Overwriting C:\Users\pabuehle\Desktop\ComputerVision\detection\notebooks\hyperdrive/train.py


### 5. Setup and run Hyperdrive experiment

#### 5.1 Create Experiment  
Experiment is the main entry point into experimenting with AzureML. To create new Experiment or get the existing one, we pass our experimentation name 'hyperparameter-tuning'.


In [46]:
exp = Experiment(workspace=ws, name='hyperparameter-tuning')

#### 5.2. Define search space

Now we define the search space of hyperparameters. To test discrete parameter values use 'choice()', and for uniform sampling use 'uniform()'. For more options, see [Hyperdrive parameter expressions](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.parameter_expressions?view=azure-ml-py).

Hyperdrive provides three different parameter sampling methods: 'RandomParameterSampling', 'GridParameterSampling', and 'BayesianParameterSampling'. Details about each method can be found [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-tune-hyperparameters). Here, we use the 'GridParameterSampling'.

In [47]:
# Grid-search
param_sampling = GridParameterSampling( {
        '--learning_rate': choice(LEARNING_RATES),
        '--max_size': choice(IM_MAX_SIZES)
    }
)

<b>AzureML Estimator</b> is the building block for training. An Estimator encapsulates the training code and parameters, the compute resources and runtime environment for a particular training scenario.
We create one for our experimentation with the dependencies our model requires as follows:

In [48]:
script_params = {
    '--data-folder': ds.as_mount(),
    '--data-subfolder': data_subfolder,
}

est = Estimator(source_directory=script_folder,
                script_params=script_params,
                compute_target=compute_target,
                entry_script='train.py',
                use_gpu=True,
                pip_packages=['nvidia-ml-py3','fastai'],
                conda_packages=['scikit-learn', 'pycocotools>=2.0','torchvision==0.3','cudatoolkit==9.0'])

We now create a HyperDriveConfig object which includes information about parameter space sampling, termination policy, primary metric, estimator and the compute target to execute the experiment runs on.

In [49]:
hyperdrive_run_config = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling=param_sampling,
    policy=None, # Do not use any early termination 
    primary_metric_name='accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=None, # Set to none to run all possible grid parameter combinations,
    max_concurrent_runs=MAX_NODES
)

#### 5.3 Run Experiment

We now run the parameter sweep and visualize the experiment progress using the `RunDetails` widget:
<img src="media/hyperdrive_widget_run.jpg" width="700px">

Once completed, the accuracy for the different runs can be analyzed via the widget, for example below is a plot of the accuracy versus learning rate below (for two different image sizes)
<img src="media/hyperdrive_widget_analysis.jpg" width="700px">


In [50]:
hyperdrive_run = exp.submit(config=hyperdrive_run_config)
print(f"Url to hyperdrive run on the Azure portal: {hyperdrive_run.get_portal_url()}")

Url to hyperdrive run on the Azure portal: https://mlworkspace.azure.ai/portal/subscriptions/989b90f7-da4f-41f9-84c9-44848802052d/resourceGroups/pabuehle_delme_hyperdrive/providers/Microsoft.MachineLearningServices/workspaces/pabuehle_ws/experiments/hyperparameter-tuning/runs/hyperparameter-tuning_1568666187755843


In [51]:
widgets.RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [ ]:
hyperdrive_run.wait_for_completion()

To load an existing Hyperdrive Run instead of start new one, we can use 
```python
hyperdrive_run = azureml.train.hyperdrive.HyperDriveRun(exp, <your-run-id>, hyperdrive_run_config=hyperdrive_run_config)
```
We also can cancel the Run with 
```python 
hyperdrive_run.cancel().
```

Once all the child-runs are finished, we can get the best run and the metrics.

In [19]:
# Get best run and print out metrics
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']
best_parameters = dict(zip(parameter_values[::2], parameter_values[1::2]))

print(f"* Best Run Id:{best_run.id}")
print(best_run)
print("\n* Best hyperparameters:")
print(best_parameters)
print(f"Accuracy = {best_run_metrics['accuracy']}")
print("Learning Rate =", best_run_metrics['learning_rate'])

AttributeError: 'NoneType' object has no attribute 'get_metrics'

In [ ]:
hyperdrive_run.get_children_sorted_by_primary_metric()

In [25]:
DATA_PATH

'C:\\Users\\pabuehle\\Desktop\\ComputerVision\\data\\odRefrigerator'

### 7. Clean up

To avoid unnecessary expenses, all resources which were created in this notebook need to get deleted once parameter search is concluded. To simplify this clean-up step, we recommended creating a new resource group to run this notebook. This resource group can then be deleted, e.g. using the Azure Portal, which will remove all created resources.

In [ ]:
# Log some outputs using scrapbook which are used during testing to verify correct notebook execution
sb.glue("best_accuracy", best_run_metrics['accuracy'])